# **Introduction**

**Here is the introduction about the exercise**

1. Import some library
2. Load data function
- Input: file_path
- Output: the data get from file (offer list get from file)
3. Filter the offer based on the requirement
- Input: the checkin_date, offer list
- Output: the list of valid offer
4. Save data
- Input: data, file_path
- Output: none
- Save the data to file_path
5. Main
- Input the checkin date, input_file_path, output file_path
- Load(input_file_path)
- Filter(checkin_date, list offer in json)
- Save(output_file_path)
6. Run

# **1. Import some library**

In [52]:
import json
from datetime import datetime, timedelta

# **2. Load data function**

In [53]:
# Đây là việc định nghĩa một hàm mới được gọi là load_json, nhận đối số file_path. Hàm này được thiết kế để đọc một tệp JSON từ đường dẫn đã cho.
def load_json(file_path):
    # Dòng này bắt đầu mở tệp ở đường dẫn đã được cung cấp trong chế độ đọc ('r'). Nó sử dụng câu lệnh with để đảm bảo rằng tài nguyên tệp sẽ được đóng tự động sau khi hoàn thành hoặc nếu có lỗi xảy ra.
    with open(file_path, 'r') as file:
        # Đoạn mã này sử dụng thư viện json của Python để đọc nội dung của tệp đã mở và lưu trữ nó vào biến data. Hàm json.load() được sử dụng để chuyển đổi dữ liệu JSON từ tệp thành cấu trúc dữ liệu Python.
        data = json.load(file)
    # Hàm load_json trả về biến data, chứa dữ liệu JSON đã được đọc từ tệp được chỉ định bởi file_path.
    return data

# **3. Filter the offer based on the requirement**

In [54]:
def filter_offers(checkin_date, offers):
    # Khởi tạo một danh sách rỗng để chứa các offer hợp lệ sau khi lọc.
    valid_offers = []
    
    # Khởi tạo một từ điển để lưu trữ thông tin về các offer theo category.
    category_dict = {}
    
    # Bắt đầu vòng lặp qua từng ưu đãi trong danh sách offers
    for offer in offers:
        # Khởi tạo danh sách rỗng để chứa các merchant hợp lệ cho mỗi offer.
        valid_merchants = []

        # Bắt đầu vòng lặp qua từng nhà merchant trong danh sách merchant của mỗi offer.
        for merchant in offer['merchants']:
            # Lấy khoảng cách của merchant đối với offer hiện tại.
            merchant_distance = merchant['distance']
            
            # Offer needs to be valid till checkin date + 5 days. (valid_to is in YYYY-MM-DD)
            # Chuyển đổi ngày hết hạn của ưu đãi từ chuỗi thành đối tượng datetime để so sánh
            valid_until = datetime.strptime(offer['valid_to'], '%Y-%m-%d')
            # Tính toán ngày hết hạn kiểm tra dựa trên ngày nhận phòng cộng thêm 5 ngày
            valid_until_checkin = valid_until + timedelta(days=5)

            # Kiểm tra xem ngày nhận phòng có nằm trong khoảng thời gian hợp lệ cho offer không ?
            if checkin_date <= valid_until_checkin.strftime('%Y-%m-%d'):
                # Only select offers with category that is Restaurant, Retail or Activity (1,2,4)
                # Kiểm tra xem category của offer có thuộc loại (1, 2 hoặc 4) không ?
                if offer['category'] in [1, 2, 4]:
                    # Nếu merchant thỏa mãn điều kiện, thêm vào danh sách merchant hợp lệ.
                    valid_merchants.append(merchant)
                 
        # Kiểm tra xem danh sách merchant hợp lệ có phần tử nào không ?   
        if valid_merchants:
            # If an offer is available in multiple merchants, only select the closest merchant
            # Sắp xếp danh sách merchant hợp lệ theo khoảng cách tăng dần
            valid_merchants = sorted(valid_merchants, key=lambda x: x['distance'])
            # Chỉ giữ merchant gần nhất trong danh sách merchant hợp lệ cho mỗi offer
            offer['merchants'] = [valid_merchants[0]]
            # Both final selected offers should be in different categories. If there are multiple offers in the same category give priority to the closest merchant offer.
            # Kiểm tra xem loại offer đã được lưu trong category_dict chưa ? Hoặc nếu có, kiểm tra xem merchant hiện tại gần với ofer hơn merchant lưu trong category_dict hơn không ?
            if offer['category'] not in category_dict or merchant_distance < category_dict[offer['category']]['distance']:
                # Lưu thông tin về offer và khoảng cách của merchant gần nhất vào category_dict theo category của offer
                category_dict[offer['category']] = {'offer': offer, 'distance': merchant_distance}

    # If there are multiple offers with different categories, select the closest merchant offers when selecting 2 offers
    selected_categories = set()
    # Bắt đầu vòng lặp qua từng thông tin về loại offer trong category_dict.
    for category_info in category_dict.values():
        # Kiểm tra xem đã chọn đủ 2 loại ưu đãi chưa ?
        if len(selected_categories) < 2:
            # Thêm loại ưu đãi vào danh sách chọn
            selected_categories.add(category_info['offer']['category'])
            # Thêm ưu đãi đã chọn vào danh sách ưu đãi hợp lệ cuối cùng
            valid_offers.append(category_info['offer'])
            
     # This class should only return 2 offers even though there are several eligible offers
     # Trả về danh sách chứa tối đa 2 ưu đãi hợp lệ được lựa chọn sau quá trình lọc.
    return valid_offers[:2]

# **4. Save data**

In [55]:
# Đây là việc định nghĩa một hàm mới có tên là save_json. Hàm này nhận hai đối số: file_path, đường dẫn đến tệp mà bạn muốn lưu dữ liệu JSON vào và data, dữ liệu cần lưu thành tệp JSON.
def save_json(file_path, data):
    # Dòng này mở tệp ở đường dẫn file_path trong chế độ ghi ('w'). Câu lệnh with đảm bảo rằng tệp sẽ được đóng tự động sau khi hoàn thành hoặc khi có lỗi xảy ra.
    with open(file_path, 'w') as file:
        # Đoạn mã này sử dụng hàm json.dump() để ghi dữ liệu JSON từ biến data vào tệp đã mở (file). Tham số indent=2 được sử dụng để thêm các khoảng trắng và thụt lề cho dễ đọc. Hàm này chuyển đổi cấu trúc dữ liệu Python trong data thành định dạng JSON và ghi nó vào tệp.
        json.dump(data, file, indent=2)

# **5. Main**

In [56]:
def main():
    # Dòng này yêu cầu người dùng nhập ngày check-in thông qua hàm input() và lưu giá trị được nhập vào biến checkin_date.
    checkin_date = input("Enter the check-in date (YYYY-MM-DD): ")
    # Yêu cầu người dùng nhập đường dẫn đến tệp input và lưu giá trị đó vào biến input_file_path
    input_file_path = input("Enter the input file path: ")
    
    # Sử dụng hàm load_json() (có thể đã được định nghĩa trước đó) để đọc dữ liệu từ tệp JSON tại đường dẫn đã nhập và lưu vào biến data.
    data = load_json(input_file_path)
    
    # Sử dụng hàm filter_offers() (có thể đã được định nghĩa ở một nơi khác) để lọc các offer từ dữ liệu đã đọc với điều kiện là ngày check-in phù hợp. Kết quả được lưu vào filtered_offers.
    filtered_offers = filter_offers(checkin_date, data['offers'])

    # Yêu cầu người dùng nhập đường dẫn đến tệp output và lưu giá trị đó vào output_file_path
    output_file_path = input("Enter the output file path: ")
    
    # Sử dụng hàm save_json() để lưu các offer đã được lọc vào một tệp JSON mới tại đường dẫn đã nhập.
    save_json(output_file_path, {'offers': filtered_offers})
    
    # In thông báo xác nhận rằng các ofer đã được lọc đã được lưu vào tệp tại đường dẫn output_file_path
    print(f"\nFiltered offers saved to {output_file_path}")

# **6. Run**

In [57]:
# Run the function
main()


Filtered offers saved to output.json
